## DGGS API dev, H3 zoning and querying

In [3]:
import yaml

**Version 0.0.4 superceded**

In [4]:
fn = "../dggs_api_server/swagger/swagger-0.0.4.yaml"

In [ ]:
with open(fn) as fh:
    read_data = yaml.load(fh, Loader=yaml.FullLoader)

# Print YAML data before sorting
# print(read_data)

In [50]:
reorder = {}

for k, v in read_data['paths'].items():
    # print(k)
    for method in ['get', 'post', 'options', 'put', 'delete', 'head']:
        # print(v['x-openapi-router-controller'])
        if method in v.keys():
            path = v[method]['x-openapi-router-controller']
            print(f"{k} [{method}]: {path}")
            reorder.update( { f"{k} [{method}]": path} )

/ [get]: dggs_api_server.controllers.capabilities_controller
/conformance [get]: dggs_api_server.controllers.capabilities_controller
/collections [get]: dggs_api_server.controllers.capabilities_controller
/collections/{collectionId}/describe [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zones [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/buffer [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/child [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/childOf [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/parentOf [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/siblingOf [get]: dggs_api_server.controllers.

**Version 0.0.6 current state**

In [6]:
fn2 = "../dggs_api_server/swagger/swagger-0.0.6.yaml"

In [7]:
with open(fn2) as fh:
    read_data2 = yaml.load(fh, Loader=yaml.FullLoader)

In [8]:
reorder2 = {}

for k, v in read_data2['paths'].items():
    # print(k)
    for method in ['get', 'post', 'options', 'put', 'delete', 'head']:
        # print(v['x-openapi-router-controller'])
        if method in v.keys():
            path = v[method]['x-openapi-router-controller'].replace("<<","").replace(">>", "_")
            print(f"{k} [{method}]: {path}")
            reorder2.update( { f"{k} [{method}]": path} )

/ [get]: dggs_api_server.controllers.capabilities_controller
/conformance [get]: dggs_api_server.controllers.capabilities_controller
/collections [get]: dggs_api_server.controllers.capabilities_controller
/collections/{collectionId}/describe [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zones [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/distance [get]: dggs_api_server.controllers.zone_query_ops_controller
/collections/{collectionId}/zone/{zoneId}/buffer [get]: dggs_api_server.controllers.zone_query_set_controller
/collections/{collectionId}/zone/{zoneId}/difference [get]: dggs_api_server.controllers.zone_query_set_controller
/collections/{collectionId}/zone/{zoneId}/intersection [get]: dggs_api_server.controllers.zone_query_set_controller
/collections/{collectionId}/zone/{zoneId}/symDifference [get]: dg

In [3]:
import requests

In [4]:
API_BASE="https://dggs-api-bozea3cspa-ew.a.run.app/dggs-api"
# API_BASE="http://localhost:8080/dggs-api"

In [5]:
# / [get]
r = requests.get(API_BASE)
r.json()

{'title': {'description': 'Access to data about Estonia via a Web API that conforms to the OGC API DGGS specification.',
  'links': [{'href': 'http://data.example.org/',
    'rel': 'self',
    'title': 'this document',
    'type': 'application/json'},
   {'href': 'http://data.example.org/api',
    'rel': 'service-desc',
    'title': 'the API definition',
    'type': 'application/vnd.oai.openapi+json;version=3.0'},
   {'href': 'http://data.example.org/api.html',
    'rel': 'service-doc',
    'title': 'the API documentation',
    'type': 'text/html'},
   {'href': 'http://data.example.org/conformance',
    'rel': 'conformance',
    'title': 'OGC API conformance classes implemented by this server',
    'type': 'application/json'},
   {'href': 'http://data.example.org/collections',
    'rel': 'data',
    'title': 'Information about the feature collections',
    'type': 'application/json'}],
  'title': 'Data in Estonia'}}

In [6]:
# /collections [get]
r = requests.get(API_BASE + "/collections")
r.json()

{'dggs-list': [{'description': '30m SRTM sampled at res 8 covering Estonia',
   'dggs-id': 'H3',
   'id': 'srtm_30m_estonia_h3',
   'links': [],
   'resolutions': ['8', '7', '6', '5', '4', '3', '2'],
   'title': 'srtm_30m_estonia_h3'}],
 'links': [{'href': {'href': 'http://data.example.org/dggs.json',
    'rel': 'self',
    'title': 'this document',
    'type': 'application/json'}},
  {'href': {'href': 'http://data.example.org/dggs.html',
    'rel': 'alternate',
    'title': 'this document as HTML',
    'type': 'text/html'}}]}

In [7]:
collection = r.json()['dggs-list'][0]
collectionId = collection['id']
print(f"working with <{collectionId}>")

working with <srtm_30m_estonia_h3>


In [8]:
# /collections/{collectionId}/describe [get]
r = requests.get(API_BASE + f"/collections/{collectionId}/describe")
r.json()

{'description': '30m SRTM sampled at res 8 covering Estonia',
 'dggs-id': 'H3',
 'id': 'srtm_30m_estonia_h3',
 'links': [],
 'resolutions': ['8', '7', '6', '5', '4', '3', '2'],
 'title': 'srtm_30m_estonia_h3'}

In [9]:
import folium
from h3 import h3

In [23]:
def visualize_polygon_features(features, color="red", folium_map=None):
    geom_ext = features[0]['geometry']['coordinates'][0]
    lng = [p[0] for p in geom_ext]
    lat = [p[1] for p in geom_ext]
    if folium_map is None:
        m = folium.Map(height=600, width=800, location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    
    style1 = {'color': '#228B22'}
    style2 = {'color': '#00FFFFFF'}
    
    for f in features:
        # folium.GeoJson(f, name=features[0]['id'],style_function=lambda x: style1 if color == 'red' else style2  ).add_to(m)
        folium.GeoJson(f, name=features[0]['id'],style_function=lambda x: { 'color': color }  ).add_to(m)
    return m

In [12]:
import copy

def dggs_json_to_geojson_feature(dggs_json):
    cell_id = dggs_json['id']
    cell_id2 = dggs_json['geometry'][0]
    assert(cell_id == cell_id2)
    geom = h3.h3_to_geo_boundary(cell_id, geo_json=True)
    geojson_feature = dggs_json.copy()
    geojson_feature['geometry'] = {'type': 'Polygon',
            'coordinates': [
                geom
            ]
        }
    return geojson_feature


In [13]:
# /collections/{collectionId}/zone [get]
# that's a weird API def artifact

r = requests.get(API_BASE + f"/collections/{collectionId}/zone")
r.json()

{'geometry': ['8808932091fffff'],
 'id': '8808932091fffff',
 'links': [],
 'properties': {'elevation': 0.0,
  'elevation_count': 1,
  'elevation_std': 0.0,
  'resolution': 8},
 'type': 'Feature'}

In [14]:
zone_geojson1 = dggs_json_to_geojson_feature(r.json())
zone_geojson1

{'geometry': {'type': 'Polygon',
  'coordinates': [((21.822582458429643, 58.22546258883676),
    (21.819530487824053, 58.221758675127255),
    (21.825011159959434, 58.21873583713209),
    (21.83354362911352, 58.219416682186775),
    (21.83659642569351, 58.22312044561261),
    (21.831115927287996, 58.226143514283564),
    (21.822582458429643, 58.22546258883676))]},
 'id': '8808932091fffff',
 'links': [],
 'properties': {'elevation': 0.0,
  'elevation_count': 1,
  'elevation_std': 0.0,
  'resolution': 8},
 'type': 'Feature'}

In [24]:
visualize_polygon_features([zone_geojson1], color="blue", folium_map=None)

In [25]:
# /collections/{collectionId}/zones [get]
query_params = {'resolution': 5, 'bbox': "25.32,58.27,26.82,58.67", 'limit': 200}
r = requests.get(API_BASE + f"/collections/{collectionId}/zones", params=query_params)
zone_json = r.json()

In [26]:
len(zone_json['features'])

19

In [27]:
zone_geojson_collection = [dggs_json_to_geojson_feature(f) for f in zone_json['features']]
m1 = visualize_polygon_features(zone_geojson_collection, color="red", folium_map=None)

In [28]:
m1

In [29]:
# /collections/{collectionId}/zones [get]
query_params = {'resolution': 6, 'bbox': "25.32,58.27,26.82,58.67", 'limit': 200}
r = requests.get(API_BASE + f"/collections/{collectionId}/zones", params=query_params)
zone_json = r.json()

In [30]:
zone_geojson_collection = [dggs_json_to_geojson_feature(f) for f in zone_json['features']]
m2 = visualize_polygon_features(zone_geojson_collection, color="blue", folium_map=m1)

In [31]:
m2

## manual H3 cells, size and resolutions for reference

In [99]:
minx, miny, maxx, maxy = [25.32, 58.27, 26.82, 58.67]
resolution = 5

area_json = {'type': 'Polygon',
            'coordinates': [
                [
                    [miny, minx],
                    [miny, maxx],
                    [maxy, maxx],
                    [maxy, minx],
                    [miny, minx]
                ]
            ]
        }
zone_id_list_from_fill = h3.polyfill(area_json, resolution)

In [100]:
len(zone_id_list_from_fill)

19

In [121]:
h3.h3_get_resolution("8511346bfffffff")

5

In [115]:
"8511340bfffffff" in zone_id_list_from_fill

True

In [103]:
zone_id_list_from_fill

{'8511340bfffffff',
 '8511341bfffffff',
 '85113443fffffff',
 '85113447fffffff',
 '8511344ffffffff',
 '85113453fffffff',
 '85113457fffffff',
 '85113463fffffff',
 '8511346bfffffff',
 '8511346ffffffff',
 '85113473fffffff',
 '8511347bfffffff',
 '851134c3fffffff',
 '851134c7fffffff',
 '851134cbfffffff',
 '851134cffffffff',
 '851134dbfffffff',
 '851136a7fffffff',
 '851136b7fffffff'}

In [101]:


m = visualize_hexagons(zone_id_list_from_fill)
display(m)

